In [5]:
using CSV
using Distances
using Clustering
using DataFrames
using MultivariateStats
using PhyloClustering

┌ Warning: Error requiring `MakieCore` from `Polynomials`
│   exception =
│    LoadError: ArgumentError: Package Polynomials does not have MakieCore in its dependencies:
│    - You may have a partially installed environment. Try `Pkg.instantiate()`
│      to ensure all packages in the environment are installed.
│    - Or, if you have Polynomials checked out for development and have
│      added MakieCore as a dependency but haven't updated your primary
│      environment's manifest file, try `Pkg.resolve()`.
│    - Otherwise you may need to report an issue with Polynomials
│    Stacktrace:
│      [1] macro expansion
│        @ .\loading.jl:1167 [inlined]
│      [2] macro expansion
│        @ .\lock.jl:223 [inlined]
│      [3] require(into::Module, mod::Symbol)
│        @ Base .\loading.jl:1144
│      [4] include(mod::Module, _path::String)
│        @ Base .\Base.jl:419
│      [5] include(x::String)
│        @ Polynomials C:\Users\82537\.julia\packages\Polynomials\6i39P\src\Polynomials.

In [13]:
function ground_true(df_1, df_2)
    a = fill(1,size(df_1)[1])
    b = fill(2,size(df_2)[1])
    gt = cat(a,b, dims = 1)
    return gt
end

function accuracy(n, gt, pred)
    matrix = confusmat(n, gt, pred)
    # Hungarian algorithm minimizes the cost, so we need to transform the matrix
    A = -matrix .+ maximum(matrix)    
    matrix = matrix[:,hungarian(A)[1]]
    x = tr(matrix)/sum(matrix)
    return matrix, x
end

accuracy (generic function with 1 method)

In [4]:
tree1 = CSV.read("../../data/trees/4_diff_topo_1_1000_11.csv", DataFrame);
tree2 = CSV.read("../../data/trees/4_diff_topo_12_1000_11.csv", DataFrame);
tree1 = Matrix(tree1);
tree2 = Matrix(tree2);

In [14]:
gt = ground_true(tree1, tree2);
tree = vcat(tree1,tree2);
tree = standardize_tree(tree);
distances = distance(tree);
M = fit(MDS, distances; maxoutdim=3, distances=true)

Classical MDS(indim = NaN, outdim = 3)

In [15]:
Y = predict(M)

3×2000 Matrix{Float64}:
 -1.37519   -0.21631    1.02305   …  -1.72528   -1.9258    -1.30154
 -0.71968   -0.782244  -1.19323       0.390588   0.78688   -2.07822
 -0.548004   0.183745   0.871203     -1.17908   -0.295854   0.180827

In [21]:
matrix = pairwise(Euclidean(), Y, dims=2)
hc = hc_label(matrix, 2)
accuracy(2, gt, hc)

([855 145; 47 953], 0.904)

In [27]:
kmeans_pred = kmeans(tree, 2)
accuracy(2, gt, kmeans_pred.assignments)

([729 271; 10 990], 0.8595)